In [0]:
-- Catalog and schema access
GRANT USAGE ON CATALOG migrate_to_dbsql TO `analysts`;
GRANT USAGE ON SCHEMA  migrate_to_dbsql.webinar TO `analysts`;

-- Table-level permissions
GRANT SELECT ON TABLE migrate_to_dbsql.webinar.iot_daily_summary TO `analysts`;
GRANT SELECT ON TABLE migrate_to_dbsql.webinar.iot_device_metrics TO `analysts`;

-- Limit write to engineering only
GRANT MODIFY ON TABLE migrate_to_dbsql.webinar.iot_device_metrics TO `data_engineers`;

-- Revoke example
REVOKE MODIFY ON TABLE migrate_to_dbsql.webinar.iot_device_metrics FROM `analysts`;

-- Access mapping: which devices each user can see
-- migrate_to_dbsql.webinar.device_access(user_email STRING, device_id STRING)

CREATE TABLE migrate_to_dbsql.webinar.device_access (
  device_id STRING,
  user_email STRING
);

INSERT INTO migrate_to_dbsql.webinar.device_access (device_id, user_email) VALUES
  ('1', 'alice@example.com'),
  ('2', 'bob@example.com'),
  ('3', 'carol@example.com'),
  ('4', 'dave@example.com'),
  ('5', 'abhijit.tilak@databricks.com'),
  ('6', 'eric.begg@databricks.com'),
  ('7', 'zachary.ryan@databricks.com'),
  ('8', 'heidi@example.com'),
  ('9', 'ivan@example.com');

CREATE OR REPLACE VIEW migrate_to_dbsql.webinar.iot_daily_summary_rls AS
SELECT s.*
FROM migrate_to_dbsql.webinar.iot_daily_summary s
JOIN migrate_to_dbsql.webinar.device_access a
  ON a.device_id = s.device_id
WHERE LOWER(a.user_email) = LOWER(current_user());

-- Grant access to the RLS view (not the base table)
GRANT SELECT ON VIEW migrate_to_dbsql.webinar.iot_daily_summary_rls TO `analysts`;

SELECT * FROM migrate_to_dbsql.webinar.iot_daily_summary_rls;


CREATE OR REPLACE VIEW migrate_to_dbsql.webinar.iot_device_metrics_cls AS
SELECT
  device_id,
  user_id,
  timestamp,
  CASE
    WHEN is_member('pii-cleared') THEN num_steps
    ELSE NULL
  END AS num_steps,
  CASE
    WHEN is_member('pii-cleared') THEN calories_burnt
    ELSE NULL
  END AS calories_burnt,
  miles_walked
FROM migrate_to_dbsql.webinar.iot_device_metrics;

GRANT SELECT ON VIEW migrate_to_dbsql.webinar.iot_device_metrics_cls TO `analysts`;

SELECT * FROM migrate_to_dbsql.webinar.iot_device_metrics_cls;

-- Region-restricted access: a user must be in the matching region group
-- migrate_to_dbsql.webinar.user_regions(user_email STRING, region STRING)

CREATE OR REPLACE VIEW migrate_to_dbsql.webinar.iot_daily_summary_abac AS
SELECT s.*
FROM migrate_to_dbsql.webinar.iot_daily_summary s
JOIN migrate_to_dbsql.webinar.user_regions ur
  ON LOWER(ur.user_email) = LOWER(current_user())
WHERE s.user_id IN (
  SELECT user_id
  FROM migrate_to_dbsql.webinar.user_regions
  WHERE region = ur.region
);

GRANT SELECT ON VIEW migrate_to_dbsql.webinar.iot_daily_summary_abac TO `analyst-group`;
